In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### Complete #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Module import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Complete
username = "aacuser"
password = "a"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dashboard')

#Complete Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
    
    html.H1('Nicholas Wood')
#Complete Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    html.Div(className = 'row',
            style = {'display' : 'flex'},
                children = [
                        html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Water Rescue'),
                        html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Mountain Rescue'),
                        html.Button(id = 'submit-button-three', n_clicks = 0, children = 'Disaster Rescue'),
                        html.Button(id = 'submit-button-four', n_clicks = 0, children = 'Reset')
                ])

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#Complete
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n_clicks'),
                Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries
            if ((int(bt1) == 0 and int(bt2) == 0 and int (bt3)== 0 and int(bt4) == 0) or 
                (int(bt4) > int(bt1) and int(bt4) > int(bt2) and int(bt4) > int(bt3))):
                df = pd.Dataframe.from_records(shelter.readAll({""}))
            elif (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4))
                df = pd.Dataframe.from_records(shelter.readAll({"animal_type": "Dog", ("breed" : "Labrador Retriever Mix" or
                                                                "breed" : "Chesapeake Bay Reteriever" or
                                                                "breed" : "Newfoundland"),
                                                                "age_upon_outcome_in_weeks" : int(156)}))
            elif (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
                df = pd.Dataframe.from_records(shelter.readAll({"animal_type": "Dog", ("breed" : "German Shepherd" or
                                                                "breed" : "Alaskan Malamute" or
                                                                "breed" : "Old English Sheepdog" or 
                                                                "breed" : "Siberian Husky" or 
                                                                "breed" : "Rottweiler"),
                                                                "age_upon_outcome_in_weeks" : int(156)}))
            elif (int(bt3) > int(bt1) and int(bt3) > int(bt2) and int(bt3) > int(bt4)):
                df = pd.Dataframe.from_records(shelter.readAll({"animal_type": "Dog", ("breed" : "Doberman Pinscher" or
                                                                "breed" : "German Shepherd" or
                                                                "breed" : "Golden Retriever" or 
                                                                "breed" : "Bloodhound" or 
                                                                "breed" : "Rottweiler"),
                                                                "age_upon_outcome_in_weeks" : int(300)}))

            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
                data=df.to_dict('records')
        
        
            return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    viewDF = pd.DataFrame.from_dict(viewdata)
    dff = viewDF.loc[rows]
    
    fig = px.pie(dff, values = dff['breed'], names = dff['breed'])
    
    return [dcc.Graph(fig)]
    
    
    

    @app.callback(
        Output('map-id', "children"),
        [Input('datatable-id', "derived_viewport_data")])
    def update_map(viewData):
    #Complete: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
            viewDF = pd.DataFrame.from_dict(viewData)

            dff = viewDF.loc[rows]

    # Austin TX is at [30.75,-97.48]
            return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and pop up
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
            ])
        ])
    ])]    
                              


app

SyntaxError: invalid syntax (<ipython-input-1-78a9021f3e21>, line 61)